In [ ]:
# Import and initialize
import datetime
import numpy as np
import pygetm

%matplotlib widget
import matplotlib.pyplot

In [ ]:
# Set up slice domain (ny=1)
domain = pygetm.domain.create_cartesian(
    500.0 * np.arange(131), 500.0 * np.arange(2), 20, lat=0, H=20, interfaces=True
)
sim = pygetm.Simulation(
    domain,
    runtype=pygetm.BAROCLINIC,
    airsea=pygetm.airsea.Fluxes(),
    log_level="ERROR",
    internal_pressure_method=pygetm.InternalPressure.BLUMBERG_MELLOR,
)

sim.temp.fill(20.0)
sim.salt.fill(np.where(domain.T.x.all_values < 32500.0, 20, 30))

# Idealized surface forcing
sim.airsea.taux.set(0.0)
sim.airsea.tauy.set(0.0)
sim.airsea.sp.set(0.0)
sim.airsea.shf.set(0.0)
sim.airsea.swr.set(0.0)

# Disable light attenuation
sim.radiation.A.set(0.0)
sim.radiation.kc1.set(0.0)
sim.radiation.kc2.set(0.0)

timestep = 15.0
ntime = 24 * int(3600.0 // timestep)
times = timestep * np.arange(ntime)
plotting_interval = 10
sim.start(datetime.datetime(2015, 1, 1), timestep=timestep, split_factor=30, report=180)

nsample_x = 2
nsample_z = 2
iy = 0
fig, ax = matplotlib.pyplot.subplots()
u = sim.momentum.pk.interp(domain.T) / domain.T.H
w = sim.momentum.ww.interp(domain.T, z=pygetm.CENTERS)
z_2d = domain.T.zc[::nsample_z, iy, ::nsample_x]
x_2d = np.broadcast_to(domain.T.x[iy, ::nsample_x], z_2d.shape)
pc = ax.pcolormesh(domain.T.xi[iy, :], domain.T.zf[:, iy, 0], sim.salt.values[:, iy, :])
Q = ax.quiver(
    x_2d,
    z_2d,
    u[::nsample_z, iy, ::nsample_x],
    w[::nsample_z, iy, ::nsample_x],
    scale=1.0,
)
title = ax.set_title("time: 0 s")


In [ ]:
for istep, time in enumerate(times):
    sim.advance()
    if istep % plotting_interval == 0:
        u = sim.momentum.pk.interp(domain.T) / domain.T.H
        sim.momentum.ww.interp(w)
        Q.set_UVC(u[::nsample_z, iy, ::nsample_x], w[::nsample_z, iy, ::nsample_x])
        pc.set_array(sim.salt.values[:, iy, :].ravel())
        title.set_text("time: %s s" % time)
        fig.canvas.draw()

sim.finish()
